## Data Preprocessing

In [1]:
import sys
sys.path.insert(0, '..')
import os

import numpy as np
import pandas as pd

In [2]:
import zipfile
with zipfile.ZipFile('./data/labeledTrainData.tsv.zip', 'r') as z:
    z.extractall('./data/')

In [3]:
data_train = pd.read_csv('./data/labeledTrainData.tsv', sep='\t')

In [4]:
print(data_train.shape)
data_train.head(5)

(25000, 3)


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [5]:
data_train['review'][0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [6]:
data_train.review.shape[0]

25000

In [7]:
from bs4 import BeautifulSoup
import re

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets 
    Every dataset is lower cased
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()

texts = []
labels = []
for idx in range(data_train.review.shape[0]):
    text = BeautifulSoup(data_train.review[idx], "lxml")
    texts.append(clean_str(text.get_text()))
    labels.append(data_train.sentiment[idx])

In [8]:
len(texts)

25000

In [9]:
texts[0]

"with all this stuff going down at the moment with mj i 've started listening to his music , watching the odd documentary here and there , watched the wiz and watched moonwalker again maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent moonwalker is part biography , part feature film which i remember going to see at the cinema when it was originally released some of it has subtle messages about mj 's feeling towards the press and also the obvious message of drugs are bad m'kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for 20 m

In [21]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten, concatenate, TimeDistributed
from keras.layers import Conv1D, MaxPooling1D, Dropout, Concatenate, SpatialDropout1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers import Bidirectional, LSTM, GRU, CuDNNLSTM, CuDNNGRU
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [11]:
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

C:\Users\sun\Anaconda3\lib\site-packages\keras_preprocessing\text.py:175: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 81988 unique tokens.


In [12]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (25000, 1000)
Shape of label tensor: (25000, 2)


In [13]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of positive and negative reviews in traing and validation set ')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

Number of positive and negative reviews in traing and validation set 
[10073.  9927.]
[2427. 2573.]


In [14]:
GLOVE_DIR = "../pre_trained word embeddings/glove.6B"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

Total 400000 word vectors in Glove 6B 100d.


In [15]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [16]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

## Attention Model

### Attention

In [17]:
# use Theano as backend
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializers.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):
        eij = K.tanh(K.dot(x, self.W))
        
        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [29]:
# use Tensorflow as backend
# https://github.com/richliao/textClassifier/issues/13#issuecomment-377323318
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializers.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):
        eij = K.tanh(K.squeeze(K.dot(x, K.expand_dims(self.W)), axis=-1))
        
        ai = K.exp(eij)
        weights = ai/K.expand_dims(K.sum(ai, axis=1),1)
        
        weighted_input = x*K.expand_dims(weights,2)
        return K.sum(weighted_input, axis=1)

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [18]:
class AttLayer(Layer):
    def __init__(self, attention_dim):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(AttLayer, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)))
        self.b = K.variable(self.init((self.attention_dim, )))
        self.u = K.variable(self.init((self.attention_dim, 1)))
        self.trainable_weights = [self.W, self.b, self.u]
        super(AttLayer, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        # size of x :[batch_size, sel_len, attention_dim]
        # size of u :[batch_size, attention_dim]
        # uit = tanh(xW+b)
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)

        ait = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            ait *= K.cast(mask, K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [18]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_gru = Bidirectional(CuDNNGRU(100, return_sequences=True))(embedded_sequences)
l_att = AttLayer(100)(l_gru)
preds = Dense(2, activation='softmax')(l_att)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - attention GRU network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

model fitting - attention GRU network
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         8198900   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1000, 200)         121200    
_________________________________________________________________
att_layer_1 (AttLayer)       (None, 200)               20200     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 8,340,702
Trainable params: 8,340,702
Non-trainable params: 0
_________________________________________________________________


C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 56s 3ms/step - loss: 0.4143 - acc: 0.7980 - val_loss: 0.3316 - val_acc: 0.8580
Epoch 2/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.2492 - acc: 0.8982 - val_loss: 0.2708 - val_acc: 0.8908
Epoch 3/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.1908 - acc: 0.9245 - val_loss: 0.2374 - val_acc: 0.9082
Epoch 4/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.1473 - acc: 0.9433 - val_loss: 0.2282 - val_acc: 0.9110
Epoch 5/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.1056 - acc: 0.9618 - val_loss: 0.2360 - val_acc: 0.9172
Epoch 6/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.0717 - acc: 0.9756 - val_loss: 0.3170 - val_acc: 0.9088
Epoch 7/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.0429 - acc: 0.9867 - val_loss: 0.3978 - val_acc:

In [19]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_gru = Bidirectional(CuDNNGRU(100, return_sequences=True))(embedded_sequences)
l_att = AttLayer(100)(l_gru)
l_dropout = Dropout(0.2)(l_att)
preds = Dense(2, activation='softmax')(l_dropout)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - attention GRU network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

model fitting - attention GRU network
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         8198900   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1000, 200)         121200    
_________________________________________________________________
att_layer_1 (AttLayer)       (None, 200)               20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 8,340,702
Trainable params: 8,340,702
Non-trainable params: 0
____________________________

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 56s 3ms/step - loss: 0.4028 - acc: 0.8090 - val_loss: 0.3140 - val_acc: 0.8760
Epoch 2/10
20000/20000 [==============================] - 55s 3ms/step - loss: 0.2531 - acc: 0.8980 - val_loss: 0.2896 - val_acc: 0.8766
Epoch 3/10
20000/20000 [==============================] - 55s 3ms/step - loss: 0.1931 - acc: 0.9250 - val_loss: 0.2176 - val_acc: 0.9130
Epoch 4/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.1511 - acc: 0.9445 - val_loss: 0.2365 - val_acc: 0.9096
Epoch 5/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.1135 - acc: 0.9600 - val_loss: 0.2374 - val_acc: 0.9120
Epoch 6/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.0811 - acc: 0.9723 - val_loss: 0.2763 - val_acc: 0.9056
Epoch 7/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.0525 - acc: 0.9831 - val_loss: 0.3241 - val_acc:

In [19]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_gru = Bidirectional(CuDNNGRU(100, return_sequences=True))(embedded_sequences)
l_att = AttLayer(100)(l_gru)
l_dropout = Dropout(0.5)(l_att)
preds = Dense(2, activation='softmax')(l_dropout)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - attention GRU network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

model fitting - attention GRU network
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         8198900   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1000, 200)         121200    
_________________________________________________________________
att_layer_2 (AttLayer)       (None, 200)               20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 402       
Total params: 8,340,702
Trainable params: 8,340,702
Non-trainable params: 0
____________________________

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 55s 3ms/step - loss: 0.2513 - acc: 0.8897 - val_loss: 0.2434 - val_acc: 0.9054
Epoch 2/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.0943 - acc: 0.9660 - val_loss: 0.2964 - val_acc: 0.9054
Epoch 3/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.0518 - acc: 0.9816 - val_loss: 0.3474 - val_acc: 0.9014
Epoch 4/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.0294 - acc: 0.9901 - val_loss: 0.4211 - val_acc: 0.8990
Epoch 5/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.0165 - acc: 0.9954 - val_loss: 0.5425 - val_acc: 0.8940
Epoch 6/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.0096 - acc: 0.9976 - val_loss: 0.6227 - val_acc: 0.9008
Epoch 7/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.0061 - acc: 0.9979 - val_loss: 0.7327 - val_acc:

In [20]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_gru = Bidirectional(CuDNNGRU(100, return_sequences=True))(embedded_sequences)
l_att = AttLayer(100)(l_gru)
l_dropout = Dropout(0.2)(l_att)
preds = Dense(2, activation='softmax')(l_dropout)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print("model fitting - attention GRU network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

model fitting - attention GRU network
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         8198900   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1000, 200)         121200    
_________________________________________________________________
att_layer_2 (AttLayer)       (None, 200)               20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 402       
Total params: 8,340,702
Trainable params: 8,340,702
Non-trainable params: 0
____________________________

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 56s 3ms/step - loss: 0.2345 - acc: 0.8959 - val_loss: 0.2277 - val_acc: 0.9122
Epoch 2/10
20000/20000 [==============================] - 55s 3ms/step - loss: 0.0684 - acc: 0.9763 - val_loss: 0.3465 - val_acc: 0.9014
Epoch 3/10
20000/20000 [==============================] - 55s 3ms/step - loss: 0.0218 - acc: 0.9934 - val_loss: 0.3927 - val_acc: 0.9014
Epoch 4/10
20000/20000 [==============================] - 55s 3ms/step - loss: 0.0095 - acc: 0.9972 - val_loss: 0.4575 - val_acc: 0.8980
Epoch 5/10
20000/20000 [==============================] - 55s 3ms/step - loss: 0.0050 - acc: 0.9985 - val_loss: 0.5368 - val_acc: 0.9026
Epoch 6/10
20000/20000 [==============================] - 55s 3ms/step - loss: 0.0032 - acc: 0.9990 - val_loss: 0.6356 - val_acc: 0.9026
Epoch 7/10
20000/20000 [==============================] - 55s 3ms/step - loss: 0.0033 - acc: 0.9988 - val_loss: 0.6061 - val_acc:

In [21]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_gru = Bidirectional(CuDNNGRU(100, return_sequences=True))(embedded_sequences)
l_att = AttLayer(100)(l_gru)
l_dropout = Dropout(0.5)(l_att)
preds = Dense(2, activation='softmax')(l_dropout)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print("model fitting - attention GRU network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=5, batch_size=50)

model fitting - attention GRU network
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         8198900   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 1000, 200)         121200    
_________________________________________________________________
att_layer_3 (AttLayer)       (None, 200)               20200     
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 402       
Total params: 8,340,702
Trainable params: 8,340,702
Non-trainable params: 0
____________________________

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 56s 3ms/step - loss: 0.1802 - acc: 0.9134 - val_loss: 0.3161 - val_acc: 0.9062
Epoch 2/5
20000/20000 [==============================] - 55s 3ms/step - loss: 0.0184 - acc: 0.9939 - val_loss: 0.3991 - val_acc: 0.9048
Epoch 3/5
20000/20000 [==============================] - 55s 3ms/step - loss: 0.0058 - acc: 0.9983 - val_loss: 0.5415 - val_acc: 0.8988
Epoch 4/5
20000/20000 [==============================] - 55s 3ms/step - loss: 9.3853e-04 - acc: 0.9998 - val_loss: 0.6161 - val_acc: 0.9032
Epoch 5/5
20000/20000 [==============================] - 55s 3ms/step - loss: 1.4262e-04 - acc: 1.0000 - val_loss: 0.6886 - val_acc: 0.9034


### Self Attention

In [17]:
from keras import backend as K
from keras.engine.topology import Layer

class Position_Embedding(Layer):
    
    def __init__(self, size=None, mode='sum', **kwargs):
        self.size = size #必须为偶数
        self.mode = mode
        super(Position_Embedding, self).__init__(**kwargs)
        
    def call(self, x):
        if (self.size == None) or (self.mode == 'sum'):
            self.size = int(x.shape[-1])
        batch_size,seq_len = K.shape(x)[0],K.shape(x)[1]
        position_j = 1. / K.pow(10000., \
                                 2 * K.arange(self.size / 2, dtype='float32' \
                               ) / self.size)
        position_j = K.expand_dims(position_j, 0)
        position_i = K.cumsum(K.ones_like(x[:,:,0]), 1)-1 #K.arange不支持变长，只好用这种方法生成
        position_i = K.expand_dims(position_i, 2)
        position_ij = K.dot(position_i, position_j)
        position_ij = K.concatenate([K.cos(position_ij), K.sin(position_ij)], 2)
        if self.mode == 'sum':
            return position_ij + x
        elif self.mode == 'concat':
            return K.concatenate([position_ij, x], 2)
        
    def compute_output_shape(self, input_shape):
        if self.mode == 'sum':
            return input_shape
        elif self.mode == 'concat':
            return (input_shape[0], input_shape[1], input_shape[2]+self.size)


class Attention(Layer):

    def __init__(self, nb_head, size_per_head, **kwargs):
        self.nb_head = nb_head
        self.size_per_head = size_per_head
        self.output_dim = nb_head*size_per_head
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.WQ = self.add_weight(name='WQ', 
                                  shape=(input_shape[0][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WK = self.add_weight(name='WK', 
                                  shape=(input_shape[1][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WV = self.add_weight(name='WV', 
                                  shape=(input_shape[2][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        super(Attention, self).build(input_shape)
        
    def Mask(self, inputs, seq_len, mode='mul'):
        if seq_len == None:
            return inputs
        else:
            mask = K.one_hot(seq_len[:,0], K.shape(inputs)[1])
            mask = 1 - K.cumsum(mask, 1)
            for _ in range(len(inputs.shape)-2):
                mask = K.expand_dims(mask, 2)
            if mode == 'mul':
                return inputs * mask
            if mode == 'add':
                return inputs - (1 - mask) * 1e12
                
    def call(self, x):
        #如果只传入Q_seq,K_seq,V_seq，那么就不做Mask
        #如果同时传入Q_seq,K_seq,V_seq,Q_len,V_len，那么对多余部分做Mask
        if len(x) == 3:
            Q_seq,K_seq,V_seq = x
            Q_len,V_len = None,None
        elif len(x) == 5:
            Q_seq,K_seq,V_seq,Q_len,V_len = x
        #对Q、K、V做线性变换
        Q_seq = K.dot(Q_seq, self.WQ)
        Q_seq = K.reshape(Q_seq, (-1, K.shape(Q_seq)[1], self.nb_head, self.size_per_head))
        Q_seq = K.permute_dimensions(Q_seq, (0,2,1,3))
        K_seq = K.dot(K_seq, self.WK)
        K_seq = K.reshape(K_seq, (-1, K.shape(K_seq)[1], self.nb_head, self.size_per_head))
        K_seq = K.permute_dimensions(K_seq, (0,2,1,3))
        V_seq = K.dot(V_seq, self.WV)
        V_seq = K.reshape(V_seq, (-1, K.shape(V_seq)[1], self.nb_head, self.size_per_head))
        V_seq = K.permute_dimensions(V_seq, (0,2,1,3))
        #计算内积，然后mask，然后softmax
        A = K.batch_dot(Q_seq, K_seq, axes=[3,3]) / self.size_per_head**0.5
        A = K.permute_dimensions(A, (0,3,2,1))
        A = self.Mask(A, V_len, 'add')
        A = K.permute_dimensions(A, (0,3,2,1))    
        A = K.softmax(A)
        #输出并mask
        O_seq = K.batch_dot(A, V_seq, axes=[3,2])
        O_seq = K.permute_dimensions(O_seq, (0,2,1,3))
        O_seq = K.reshape(O_seq, (-1, K.shape(O_seq)[1], self.output_dim))
        O_seq = self.Mask(O_seq, Q_len, 'mul')
        return O_seq
        
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], self.output_dim)

In [20]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

#embedded_sequences = Position_Embedding()(embedded_sequences) #增加Position_Embedding能轻微提高准确率
O_seq = Attention(8,16)([embedded_sequences, embedded_sequences, embedded_sequences])
O_seq = GlobalAveragePooling1D()(O_seq)
O_seq = Dropout(0.2)(O_seq)
preds = Dense(2, activation='softmax')(O_seq)

model = Model(sequence_input, preds)

model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=5, batch_size=16)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1000, 100)    8198900     input_3[0][0]                    
__________________________________________________________________________________________________
attention_3 (Attention)         (None, 1000, 128)    38400       embedding_1[2][0]                
                                                                 embedding_1[2][0]                
                                                                 embedding_1[2][0]                
__________________________________________________________________________________________________
global_ave

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 886s 44ms/step - loss: 0.3783 - acc: 0.8212 - val_loss: 0.2826 - val_acc: 0.8830
Epoch 2/5
20000/20000 [==============================] - 882s 44ms/step - loss: 0.2131 - acc: 0.9169 - val_loss: 0.2584 - val_acc: 0.8940
Epoch 3/5
20000/20000 [==============================] - 883s 44ms/step - loss: 0.1257 - acc: 0.9531 - val_loss: 0.3041 - val_acc: 0.8830
Epoch 4/5
20000/20000 [==============================] - 898s 45ms/step - loss: 0.0657 - acc: 0.9777 - val_loss: 0.4544 - val_acc: 0.8784
Epoch 5/5
20000/20000 [==============================] - 893s 45ms/step - loss: 0.0302 - acc: 0.9904 - val_loss: 0.6209 - val_acc: 0.8658


In [21]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

embedded_sequences = Position_Embedding()(embedded_sequences) #增加Position_Embedding能轻微提高准确率
O_seq = Attention(8,16)([embedded_sequences, embedded_sequences, embedded_sequences])
O_seq = GlobalAveragePooling1D()(O_seq)
O_seq = Dropout(0.2)(O_seq)
preds = Dense(2, activation='softmax')(O_seq)

model = Model(sequence_input, preds)

model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=3, batch_size=16)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1000, 100)    8198900     input_4[0][0]                    
__________________________________________________________________________________________________
position__embedding_1 (Position (None, 1000, 100)    0           embedding_1[3][0]                
__________________________________________________________________________________________________
attention_4 (Attention)         (None, 1000, 128)    38400       position__embedding_1[0][0]      
                                                                 position__embedding_1[0][0]      
          

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 20000 samples, validate on 5000 samples
Epoch 1/3
20000/20000 [==============================] - 898s 45ms/step - loss: 0.2377 - acc: 0.8946 - val_loss: 0.3168 - val_acc: 0.8806
Epoch 2/3
20000/20000 [==============================] - 896s 45ms/step - loss: 0.0800 - acc: 0.9705 - val_loss: 0.5898 - val_acc: 0.8410
Epoch 3/3
20000/20000 [==============================] - 888s 44ms/step - loss: 0.0334 - acc: 0.9878 - val_loss: 0.6799 - val_acc: 0.8574


### HRNN

In [17]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from nltk import tokenize

MAX_SENT_LENGTH = 100
MAX_SENTS = 15

reviews = []
labels = []
texts = []

for idx in range(data_train.review.shape[0]):
    text = BeautifulSoup(data_train.review[idx], "lxml")
    text = clean_str(text.get_text())
    texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    reviews.append(sentences)
    labels.append(data_train.sentiment[idx])

data = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

for i, sentences in enumerate(reviews):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
                if k < MAX_SENT_LENGTH and tokenizer.word_index[word] < MAX_NB_WORDS:
                    data[i, j, k] = tokenizer.word_index[word]
                    k = k + 1

word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of positive and negative reviews in traing and validation set')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

Total 81988 unique tokens.
Shape of data tensor: (25000, 15, 100)
Shape of label tensor: (25000, 2)
Number of positive and negative reviews in traing and validation set
[ 9998. 10002.]
[2502. 2498.]


In [18]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SENT_LENGTH,
                            trainable=True,
                            mask_zero=True)

In [22]:
sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(GRU(100))(embedded_sequences)
sentEncoder = Model(sentence_input, l_lstm)

review_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(GRU(100))(review_encoder)
preds = Dense(2, activation='softmax')(l_lstm_sent)
model = Model(review_input, preds)

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
model.summary()
print("model fitting - Hierachical attention network")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 15, 100)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 15, 200)           8319500   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 200)               180600    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 8,500,502
Trainable params: 8,500,502
Non-trainable params: 0
_________________________________________________________________
model fitting - Hierachical attention network


C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 125s 6ms/step - loss: 0.5248 - acc: 0.7323 - val_loss: 0.4053 - val_acc: 0.8270
Epoch 2/10
20000/20000 [==============================] - 121s 6ms/step - loss: 0.3296 - acc: 0.8594 - val_loss: 0.4344 - val_acc: 0.8120
Epoch 3/10
20000/20000 [==============================] - 122s 6ms/step - loss: 0.2492 - acc: 0.8996 - val_loss: 0.3192 - val_acc: 0.8744
Epoch 4/10
20000/20000 [==============================] - 122s 6ms/step - loss: 0.1835 - acc: 0.9298 - val_loss: 0.3468 - val_acc: 0.8696
Epoch 5/10
20000/20000 [==============================] - 122s 6ms/step - loss: 0.1294 - acc: 0.9527 - val_loss: 0.3730 - val_acc: 0.8686
Epoch 6/10
20000/20000 [==============================] - 124s 6ms/step - loss: 0.0801 - acc: 0.9729 - val_loss: 0.4922 - val_acc: 0.8670
Epoch 7/10
20000/20000 [==============================] - 124s 6ms/step - loss: 0.0423 - acc: 0.9864 - val_loss: 0.5912 - v

### HATT

In [34]:
class AttLayer(Layer):
    def __init__(self, attention_dim):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(AttLayer, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)))
        self.b = K.variable(self.init((self.attention_dim, )))
        self.u = K.variable(self.init((self.attention_dim, 1)))
        self.trainable_weights = [self.W, self.b, self.u]
        super(AttLayer, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        # size of x :[batch_size, sel_len, attention_dim]
        # size of u :[batch_size, attention_dim]
        # uit = tanh(xW+b)
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)

        ait = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            ait *= K.cast(mask, K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [49]:
# use Tensorflow as backend
# https://github.com/richliao/textClassifier/issues/13#issuecomment-377323318
class AttLayer2(Layer):
    def __init__(self, **kwargs):
        self.init = initializers.get('normal')
        self.supports_masking = True
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer2, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer2, self).build(input_shape)  # be sure you call this somewhere!
        
    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        eij = K.tanh(K.squeeze(K.dot(x, K.expand_dims(self.W)), axis=-1))
        
        ai = K.exp(eij)
        
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            ai *= K.cast(mask, K.floatx())
        
        weights = ai/K.expand_dims(K.sum(ai, axis=1),1)
        
        weighted_input = x*K.expand_dims(weights,2)
        return K.sum(weighted_input, axis=1)

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [54]:
import keras.backend as K
from keras.layers import Layer
from keras import initializers, regularizers, constraints

def dot_product(x, kernel):
    """
    https://github.com/richliao/textClassifier/issues/13#issuecomment-377323318
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    
class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    
    Note: The layer has been tested with Keras 2.0.6
    
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """
    
    def __init__(self, return_coefficients=False,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.return_coefficients = return_coefficients
        self.init = initializers.get('glorot_uniform')
        
        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        
        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)
        
        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)
    
    def build(self, input_shape):
        assert len(input_shape) == 3
        
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        
        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)
        
        super(AttentionWithContext, self).build(input_shape)
    
    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None
    
    def call(self, x, mask=None):
        uit = dot_product(x, self.W)
        
        if self.bias:
            uit += self.b
        
        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)
        
        a = K.exp(ait)
        
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())
        
        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        
        a = K.expand_dims(a)
        weighted_input = x * a
        
        if self.return_coefficients:
            return [K.sum(weighted_input, axis=1), a]
        else:
            return K.sum(weighted_input, axis=1)
    
    
    
    def compute_output_shape(self, input_shape):
        if self.return_coefficients:
            return [(input_shape[0], input_shape[-1]), (input_shape[0], input_shape[-1], 1)]
        else:
            return input_shape[0], input_shape[-1]


In [58]:
MAX_SENTS = 15
MAX_SENT_LENGTH =100

sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences)
l_att = AttLayer(100)(l_lstm)
sentEncoder = Model(sentence_input, l_att)

review_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(GRU(100, return_sequences=True))(review_encoder)
l_att_sent = AttentionWithContext(100)(l_lstm_sent)
preds = Dense(2, activation='softmax')(l_att_sent)
model = Model(review_input, preds)

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
model.summary()
print("model fitting - Hierachical attention network")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=1, batch_size=50)

ValueError: Dimensions must be equal, but are 15 and 100 for 'attention_with_context_3/mul' (op: 'Mul') with input shapes: [?,15], [?,15,100].

In [35]:
from keras.layers import TimeDistributed

sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences)
l_dense = TimeDistributed(Dense(200))(l_lstm)
l_att = AttLayer(10)(l_dense)
sentEncoder = Model(sentence_input, l_att)

review_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(GRU(100, return_sequences=True))(review_encoder)
l_dense_sent = TimeDistributed(Dense(200))(l_lstm_sent)
l_att_sent = AttLayer(10)(l_dense_sent)
preds = Dense(2, activation='softmax')(l_att_sent)
model = Model(review_input, preds)

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.summary()

print("model fitting - Hierachical attention network")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 15, 100)           0         
_________________________________________________________________
time_distributed_15 (TimeDis (None, 15, 200)           8313020   
_________________________________________________________________
bidirectional_19 (Bidirectio (None, 15, 200)           180600    
_________________________________________________________________
time_distributed_16 (TimeDis (None, 15, 200)           40200     
_________________________________________________________________
att_layer_16 (AttLayer)      (None, 200)               2020      
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 402       
Total params: 8,536,242
Trainable params: 8,536,242
Non-trainable params: 0
_________________________________________________________________


C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 20000 samples, validate on 5000 samples
Epoch 1/10


InvalidArgumentError: Inputs to operation bidirectional_19/while/Select_1 of type Select must have the same size and shape.  Input 0: [50,10000] != input 1: [50,100]
	 [[Node: bidirectional_19/while/Select_1 = Select[T=DT_FLOAT, _class=["loc:@training_4/RMSprop/gradients/AddN_7"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](bidirectional_19/while/Tile, bidirectional_19/while/add_6, bidirectional_19/while/Switch_3:1)]]
	 [[Node: loss_4/mul/_1385 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_6143_loss_4/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]